In [1]:
from pathlib import Path
import stackstac
import xarray as xr
import glob
import shutil
import pystac
from itertools import groupby
from shapely.geometry import Polygon, mapping
import rasterio as rio
import os
import pandas as pd
from rasterio.warp import transform_bounds, transform_geom
from pystac.extensions import item_assets 
from pystac.extensions.projection import AssetProjectionExtension
from pystac import (Catalog, CatalogType)


In [2]:
import sys
sys.path.insert(0, '/uufs/chpc.utah.edu/common/home/u1269862/2023/expanding_retreat/expanding_retreat/')

import retreat_setup_tools


# RETREAT: Build STAC catalog for new region of data

4/11 end of day --- having an issue where the assets are being created in the stac catalog (can see them in the in-memory representation) but are not being written to file. not suer why???? giving up for now

4/12
-- the posix path from the dis_angs / dis-mags etc is missing the footprint directory in the path. i think this could be causing the assets to not be saved? their hrefs are wrong? waiting for download to finish, then will delete files not in subdirs (i think they're all duplicates) then try again

In [3]:
#set path to where data is
path = '/uufs/chpc.utah.edu/common/home/cryosphere/emarshall/retreat_itslive_work/raw_data/412_retreat/'


The first time doing this, the files in the `raw_data/new_retreat/` dir will need to be organized into footprints. do this with the retreat_setup_tools.data_directory_setup() function:

In [29]:
print(f'{path}{tile}')
# tile is a single footprint of  retreat data

/uufs/chpc.utah.edu/common/home/cryosphere/emarshall/retreat_itslive_work/raw_data/412_retreat/10_012_014


In [31]:
folder = Path(path)
files_ls = os.listdir(folder)
#make list of all files in dir
files_ls = [x for x in files_ls if 'filelist_velocities' not in x]
#find unique footprints in the files list
footprint_ids = set([x[:10] for x in files_ls])
#print(len(footprint_ids))
    
#make subdirectories for each footprint
for tile in footprint_ids:
    #print(tile)
    full_path = os.path.join(path, tile)
    #print(full_path)
    os.makedirs(full_path, exist_ok=True)
    #move_files_to_tile_subdir(path, tile)
    for data in glob.glob(f'{path}{tile}*'):
        #print(data)
        #print(tile)
        shutil.move(data, f'{path}/{tile}')

In [4]:
retreat_setup_tools.data_directory_setup(path)

70


In [6]:
folder = Path(path)

## Make STAC catalog and items

In [7]:
#each of the RETREAT data vars
dis_angs = list(folder.rglob('*-dis_angle*.tif'))
dis_azs = list(folder.rglob('*-dis_az*.tif'))
dis_mags = list(folder.rglob('*-dis_mag*.tif'))
dis_N_angs = list(folder.rglob('*dis_N_ang*.tif'))
dis_rs = list(folder.rglob('*dis_r*.tif'))

In [8]:
dis_mags[0]

PosixPath('/uufs/chpc.utah.edu/common/home/cryosphere/emarshall/retreat_itslive_work/raw_data/412_retreat/10_048_001/10_048_001_20141102-dis_mag+S1_20141021T000238_4B47_1_10_1_10_1_10_S1_20141114T000237_0DDF_1_10_1_10_1_10+250-50_50-10_0.00-0.08_2_geo_filtered_corrected.tif')

In [9]:
print(len(dis_mags))

0
1750


In [10]:
#create STAC catalog
catalog = pystac.Catalog(
    id = 'RETREAT data, eastern',
    description = 'RETREAT velocity data downloaded for eastern himalaya footprint',
    stac_extensions = ['https://stac-extensions.github.io/projection/v1.0.0/schema.json']
)

In [11]:
retreat_setup_tools.build_stac_catalog_single_var(catalog, dis_mags, 'dis_mag')

1750


In [12]:
catalog

"ID: RETREAT data, eastern"
Description: RETREAT velocity data downloaded for eastern himalaya footprint
https://stac-extensions.github.io/projection/v1.0.0/schema.json
ID: 10_048_001_20141102
"Bounding Box: (87.10456233879152, 26.2978468357303, 90.11048969939641, 28.55473136214796)"
Datetime: 2014-11-02 00:00:00
tile: 10_048_001
sensor: S1
img1_date: 20141021T000238
img1_id: 4B47
img2_date: 20141114T000237


In [13]:
retreat_setup_tools.add_asset_to_stac_catalog(catalog, dis_mags, 'dis_mag')

1750


In [23]:
catalog

"ID: RETREAT data, eastern"
Description: RETREAT velocity data downloaded for eastern himalaya footprint
https://stac-extensions.github.io/projection/v1.0.0/schema.json
ID: 10_004_001_20141110
"Bounding Box: (93.51272966157187, 27.49000025842783, 96.55102284540982, 29.745738867847376)"
Datetime: 2014-11-10 00:00:00
tile: 10_004_001
sensor: S1
img1_date: 20141017T233737
img1_id: 5D8F
img2_date: 20141204T233736


In [14]:
catalog.normalize_hrefs('/uufs/chpc.utah.edu/common/home/u1269862/2023/expanding_retreat/retreat_stac_catalog')


In [15]:
catalog.save(catalog_type=CatalogType.SELF_CONTAINED)
